In [74]:
import sys
sys.path.append('../')

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv('env'), override=True)

from engine.weaviate_interface_v4 import WeaviateIndexer, WeaviateWCS, get_weaviate_client

#standards
import os
import time
import json
from typing import List
from tqdm import tqdm
from rich import print  # nice library that provides improved printing output (overrides default print function)
from rich.pretty import pprint
import pandas as pd


In [3]:
datadir = "../../data"
pfile = os.path.join(datadir, 'text_vectors.parquet')

In [75]:
#read env vars from local .env file
api_key = os.environ['FINRAG_WEAVIATE_API_KEY']
url = os.environ['FINRAG_WEAVIATE_ENDPOINT']
model_path = 'sentence-transformers/all-mpnet-base-v2'
# careful this one is 768 bits long, all_Mini-L6-v2 is 384 bits
#instantiate client, 
try:
    client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path)
    client._client.is_live()
    print(f"Client is ready {client._client.is_ready()}")
except Exception as e:
    print(e)
    
# Alternatively we can simply use the get_weaviate_client 
# convenice function, which assumes a default config
# client = get_weaviate_client()

/Volumes/DATA/Dropbox/IMAC_BACKUP/WORK/PROJECTS/LIQUIDITY/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Client is ready True

In [71]:
client._client.is_live()

False

In [67]:
client._connect()

In [45]:
from weaviate.classes.config import Property, DataType

finrag_properties = [  
                Property(name='filename',
                         data_type=DataType.TEXT,
                         description='Name of the file',
                         index_filterable=True,
                         index_searchable=True),
                # Property(name='keywords',
                #          data_type=DataType.TEXT_ARRAY,
                #          description='Keywords associated with the file',
                #          index_filterable=True,
                #          index_searchable=True),
                # Property(name='summary',
                #          data_type=DataType.TEXT,
                #          description='Short summary of the article',
                #          index_filterable=True,
                #          index_searchable=True),
                Property(name='content',
                         data_type=DataType.TEXT,
                          description='Splits of the article',
                         index_filterable=True,
                         index_searchable=True),
              ]

In [46]:
class_name = "FinRag_mpnet-base-v2"

class_config = {'classes': [

                      {"class": class_name,        
                       
                       "description": "Financial reports", 
                       
                       "vectorIndexType": "hnsw", 
                       
                       # Vector index specific settings
                       "vectorIndexConfig": {                   
                          
                            "ef": 64,
                            "efConstruction": 128, 
                            "maxConnections": 32,    
                                            },
                       
                       "vectorizer": "none",            
                       
                       # pre-defined property mappings
                       "properties": finrag_properties }         
                      ]
               }
print(class_config)

{
    'classes': [
        {
            'class': 'FinRag_mpnet-base-v2',
            'description': 'Financial reports',
            'vectorIndexType': 'hnsw',
            'vectorIndexConfig': {'ef': 64, 'efConstruction': 128, 'maxConnections': 32},
            'vectorizer': 'none',
            'properties': [
                Property(
                    name='filename',
                    dataType=<DataType.TEXT: 'text'>,
                    description='Name of the file',
                    indexFilterable=True,
                    indexSearchable=True,
                    nestedProperties=None,
                    skip_vectorization=False,
                    tokenization=None,
                    vectorize_property_name=True
                ),
                Property(
                    name='content',
                    dataType=<DataType.TEXT: 'text'>,
                    description='Splits of the article',
                    indexFilterable=True,
                    indexSearchable=True,
                    nestedProperties=None,
                    skip_vectorization=False,
                    tokenization=None,
                    vectorize_property_name=True
                )
            ]
        }
    ]
}

In [ ]:
collection_name='finrag'
client.create_collection(collection_name=collection_name, 
                         properties=finrag_properties, 
                         description='Financial reports')

Collection "finrag" created


In [48]:
client.delete_collection(collection_name='finrag')

Collection "finrag" deleted


In [76]:
client.show_all_collections()

['Finrag']

In [77]:
client.show_collection_config('Finrag')

_CollectionConfig(name='Finrag', description='Financial reports', generative_config=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False), properties=[_Property(name='filename', description='Name of the file', data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none'), _Property(name='content', description='Splits of the article', data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none')], references=[], replication_config=_ReplicationConfig(facto

In [78]:
client.show_collection_properties('Finrag')

[_Property(name='filename', description='Name of the file', data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none'),
 _Property(name='content', description='Splits of the article', data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none')]

In [51]:
indexer = WeaviateIndexer(client)

In [79]:
data = pd.read_parquet(pfile)
data.file.value_counts()

FileNotFoundError: [Errno 2] No such file or directory: '../../data/text_vectors.parquet'

In [53]:
batch_object = indexer.batch_index_data(data.to_dict('records'), collection_name, 256)

100%|██████████| 41/41 [00:00<00:00, 853.70it/s]


Processing finished in 0.01 minutes.
Batch job completed with zero errors.


In [18]:
type(batch_object)

dict

In [54]:
list(batch_object.keys())

['num_errors', 'error_messages', 'doc_ids']

In [55]:
data.head()

,file,content,content_embedding
0,AMZN_Moodys_CreditRating_2023.pdf,CORPORATES\nCREDIT OPINION\n23 May 2023\nUpdat...,"[-0.009635023772716522, 0.01748381368815899, -..."
1,AMZN_Moodys_CreditRating_2023.pdf,The company is reliant\non the operating incom...,"[0.0033761824015527964, 0.026259412989020348, ..."
2,AMZN_Moodys_CreditRating_2023.pdf,Exhibit 1\nAmazon's debt has continued to rise...,"[-0.01308224257081747, 0.06867944449186325, -0..."
3,AMZN_Moodys_CreditRating_2023.pdf,MOODY'S INVESTORS SERVICE CORPORATES\nCredit s...,"[-0.009228247217833996, 0.023773541674017906, ..."
4,AMZN_Moodys_CreditRating_2023.pdf,Factors that could lead to a downgrade\nRating...,"[-0.027790367603302002, 0.01033176202327013, -..."


In [56]:
data.to_dict('records')

[{'file': 'AMZN_Moodys_CreditRating_2023.pdf',
  'content': "CORPORATES\nCREDIT OPINION\n23 May 2023\nUpdate\nRATINGS\nAmazon.com, Inc.\nDomicile Seattle, Washington,\nUnited States\nLong Term Rating A1\nType Senior Unsecured -\nDom Curr\nOutlook Stable\nPlease see the ratings section  at the end of this report\nfor more information. The ratings and outlook shown\nreflect information as of the publication date.\nContacts\nChristina Boni +1.212.553.0514\nSenior Vice President\nchristina.boni@moodys.com\nJack Myers +1.212.553.5116\nAssociate Analyst\njack.myers@moodys.com\nMargaret Taylor +1.212.553.0424\nAssociate Managing Director\nmargaret.taylor@moodys.comAmazon.com, Inc.\nUpdate to credit analysis\nSummary\nAmazon.com, Inc. 's (A1/Prime-1 stable) credit profile reflects its powerful global brand, which\nis synonymous with online retail, as well as the strength and profitability of Amazon Web\nServices (“AWS”), the market leader in the cloud computing market.",
  'content_embedding':

In [57]:
query = "What is Amazon debt?"
response = client.keyword_search(request=query,
                                 collection_name=collection_name,
                                 query_properties=['content'], 
                                 limit=5,
                                 filter=None,  
                                 return_properties=['filename', 'content'],
                                 return_raw=False)  # true gives a lot of additional info (not useful for RAG)

print(response)

[
    {
        'content': "Exhibit 1\nAmazon's debt has continued to rise as operating income remains below 
2019\n$0$20,000$40,000$60,000$80,000$100,000$120,000$140,000$160,000$180,000\n$0$5,000$10,000$15,000$20,000$25,000$
30,000\n2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 Q1 -23 LTM\nMoody's Adjusted Debt (USD Millions)Moody's 
Adj. Operating Income (USD Millions)Moody's adjusted operating income Moody's adjusted debt\nDebt includes 
lease\nSource: Moody’s Financial Metrics™",
        'filename': None,
        'score': 0.8038355112075806
    },
    {
        'content': "Our estimates assumes that free cash flow is positive and utilized to\nreduce funded debt. 
Nonetheless, the economic back drop remains weak which poses a need for continued capital allocation discipline\nto
achieve this goal. We also recognize that cash+short term investments as a percentage of debt remains below 
historical levels.\nExhibit 5\nRetained cash flow to debt expected to recover in the next 12-18 
months\n20%25%30%35%40%45%50%55%60%65%70%\n2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 Q1-23 LTM 12-18 
Months\nForward ViewUp Trigger Down Trigger RCF/Debt\nCredit metrics reflect Moody's standard adjustments\nSource: 
Moody’s Financial Metrics™, Moody's estimates\nESG considerations\nAmazon.com, Inc.",
        'filename': None,
        'score': 0.7088446021080017
    },
    {
        'content': "MOODY'S INVESTORS SERVICE CORPORATES\ncontent which is a key component to its Prime's value 
proposition and differentiates its offering beyond shipping. MGM's revenue of\nabout $1.5 billion represents less 
than 1% of Amazon's revenue. The company also closed in Q1 2023 on its $4 billion purchase of\n1Life Healthcare 
Inc. “One Medical,” a national primary care provider with approximately $1 billion of LTM revenue as of 
December\n2022. One Medical operates a chain of primary healthcare clinics. The business is based on a membership 
model where the company\ncharges a fixed monthly subscription fee and in exchange provides regular primary care 
services. The company focuses on both physical\nappointments and digital offerings. The company also has announced 
its intention to acquire iRobot for approximately $1.9 billion. The\npurchase is currently still being reviewed by 
the FTC.\nWe estimate that Amazon's RCF/debt can return to near our target of 50% RCF/debt at the end of 2023 to 
the extent that Amazon\nprioritizes aligning free cash flow generation with its investments.",
        'filename': None,
        'score': 0.6756844520568848
    },
    {
        'content': "MOODY'S INVESTORS SERVICE CORPORATES\nCredit strengths\n»Leading online retailer and cloud 
provider\n»AWS provides an increasing income stream provides solid positioning in AI\n»Prime membership base 
supports customer loyalty\n»Advertising revenue remains a significant area of growth\nCredit 
challenges\n»Brick-and-mortar retailers continue to increase online retail presence\n»Heightened cloud competition 
from larger, well capitalized tech companies\n»Inefficiencies in its retail fulfillment operations weigh on 
profitability\n»Reduced cash balance lowers cushion for volatile or heavy investment periods\nRating outlook\nThe 
stable outlook reflects our view that Amazon will quickly restore its credit metrics to levels reflective of its A1
rating. The outlook\nalso assumes that Amazon will maintain excellent liquidity and consistent financial 
strategies.\nFactors that could lead to an upgrade\n»Ratings could be upgraded if Amazon's numerous investments 
generate commensurate levels of profitability such that RCF/debt is\nmaintained around 65%.\n»Additional factors 
that would be critical for an upgrade are the continued maintenance of very strong liquidity, a robust cash 
and\ninvestments to debt position and maintenance of conservative financial strategies.",
        'filename': None,
        'score': 0.6546109914779663
    },
    {
        'content': "The company is reliant

In [58]:
vector_response = client.vector_search(request=query,
                                       collection_name=collection_name,
                                       limit=5, 
                                       return_properties=['filename', 'content'],
                                       filter=None,
                                       return_raw=False,
                                       device='cpu')
print(vector_response)

[
    {
        'content': "Exhibit 1\nAmazon's debt has continued to rise as operating income remains below 
2019\n$0$20,000$40,000$60,000$80,000$100,000$120,000$140,000$160,000$180,000\n$0$5,000$10,000$15,000$20,000$25,000$
30,000\n2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 Q1 -23 LTM\nMoody's Adjusted Debt (USD Millions)Moody's 
Adj. Operating Income (USD Millions)Moody's adjusted operating income Moody's adjusted debt\nDebt includes 
lease\nSource: Moody’s Financial Metrics™",
        'filename': None,
        'distance': 0.2852347493171692
    },
    {
        'content': "The use of proceeds were for general corporate purposes. We expect\npositive free cash flow to 
be applied to debt reduction supported by improved working capital and better profitability at its 
non-AWS\noperations.\nIn March 2022, the company expanded its US commercial paper program to $20 billion. The 
commercial paper program includes the\noption to issue €3 billion and is backed by a $10 billion revolving credit 
facility expiring March 29, 2025 as well as a $10 billion 364-day\ncredit facility which was put in place on 
November 18, 2022 and may be extended once. As of March 31, 2023 the company had $7.8\nbillion of commercial paper 
outstanding under its programs. A significant credit consideration to its short term and long term ratings\nis 
Amazon's commitment to fully cover all commercial paper balances with the availability under its committed 
revolvers and excess\nsame-day available cash balances. The commercial paper program is likely to be used to bridge
working capital swings and adds to its\nformidable liquidity profile. The revolving credit facility has same day 
availability, no ongoing MAC clause and no financial covenants.\n6          23 May 2023 Amazon.com, Inc.",
        'filename': None,
        'distance': 0.31232476234436035
    },
    {
        'content': "The company is reliant\non the operating income derived from AWS, as its non-AWS profitability 
has remained weak\nsince the end of 2021. Although the company is making progress with improving productivity\nand 
reducing costs, online operating margins remain well below historical levels. Amazon has\ntaken actions to make its
fulfillment operations more efficient as its business grows into its\ncapacity, which doubled during the pandemic. 
Amazon has also built a solid ecosystem of\nentertainment content that enhances its offering, operates a formidable
third-party seller\nbusiness and generates a solid and growing revenue stream from advertising. Nonetheless,\nits 
credit metrics are currently weak for the A1 rating with RCF/Debt below 50%, as lower\nprofitability, coupled with 
increased levels of investment have led to higher debt levels\nand lower cash balances. Capital allocation will be 
critical to improving its credit profile as\nAmazon navigates a weaker economic backdrop that could dampen demand 
for its products\nand services as it pursues cost reductions and efficiencies to restore profitability at 
online\nretail. The growing online presence of brick-and-mortar retailers, as well as the increasing\ncompetition 
from larger, well capitalized companies in AWS' universe also presents future\nchallenges.",
        'filename': None,
        'distance': 0.3435631990432739
    },
    {
        'content': "MOODY'S INVESTORS SERVICE CORPORATES\nAppendix\nExhibit 10\n(in US 
Millions)FYE\nDec-18FYE\nDec-19FYE\nDec-20FYE\nDec-21FYE\nDec-22LTM Ending\nMar-23\nAs Reported Debt49,289 24,719 
32,971 50,235 70,149 69,084\nOperating Leases 21,442 52,814 67,533 82,083 84,823 85,695\nNon-Standard Adjustments 
101 101 725 725 6,800 15,800\nMoody's-Adjusted Debt 70,832 77,634 101,229 133,043 161,772 170,579Moody's-Adjusted 
Debt Breakdown\nAmazon.com, Inc.\nAll figures are calculated using Moody’s estimates and standard adjustments.",
        'filename': None,
        'distance': 0.34609949588775635
    },
    {
        'content': "Its business diversification with AWS and its

In [80]:
resp = client.hybrid_search(
            request=query,
            collection_name=collection_name,
            alpha=0.5,  # higher = more like keyword search 
            limit=5, 
            return_properties=['filename', 'content'],
            filter=None,
            return_raw=False,
            device='cpu')

pprint(resp)

[
│   {
│   │   'content': 'UTC\nFinancial Leverage (Reporting Currency) Actual Forecast\nFiscal Year, ends 31 Dec 2020 2021 2022 2023 2024 2025 2026 2027\nDebt/Capital % 49.3 70.0 58.3 55.3 53.6 51.2 48.8 47.2\nAssets/Equity 3.3 5.6 4.1 3.8 3.6 3.4 3.2 3.0\nNet Debt/EBITDA 2.8 3.5 9.6 2.7 2.6 2.4 2.3 2.1\nTotal Debt/EBITDA 3.9 4.4 10.1 3.1 3.1 2.9 2.8 2.7\nEBITDA/ Net Interest Expense 5.1 7.6 2.2 7.2 7.4 7.8 8.1 8.',
│   │   'filename': None,
│   │   'score': 0.5773178935050964
│   },
│   {
│   │   'content': 'UTC\nFinancial Leverage (Reporting Currency) Actual Forecast\nFiscal Year, ends 31 Dec 2020 2021 2022 2023 2024 2025 2026 2027\nDebt/Capital % 49.3 70.0 58.3 55.3 53.6 51.2 48.8 47.2\nAssets/Equity 3.3 5.6 4.1 3.8 3.6 3.4 3.2 3.0\nNet Debt/EBITDA 2.8 3.5 9.6 2.7 2.6 2.4 2.3 2.1\nTotal Debt/EBITDA 3.9 4.4 10.1 3.1 3.1 2.9 2.8 2.7\nEBITDA/ Net Interest Expense 5.1 7.6 2.2 7.2 7.4 7.8 8.1 8.',
│   │   'filename': None,
│   │   'score': 0.5773178935050964
│   },
│   {
│   │   'content': 'UTC\nFinancial Leverage (Reporting Currency) Actual Forecast\nFiscal Year, ends 31 Dec 2020 2021 2022 2023 2024 2025 2026 2027\nDebt/Capital % 49.3 70.0 58.3 55.3 53.6 51.2 48.8 47.2\nAssets/Equity 3.3 5.6 4.1 3.8 3.6 3.4 3.2 3.0\nNet Debt/EBITDA 2.8 3.5 9.6 2.7 2.6 2.4 2.3 2.1\nTotal Debt/EBITDA 3.9 4.4 10.1 3.1 3.1 2.9 2.8 2.7\nEBITDA/ Net Interest Expense 5.1 7.6 2.2 7.2 7.4 7.8 8.1 8.',
│   │   'filename': None,
│   │   'score': 0.5773178935050964
│   },
│   {
│   │   'content': 'UTC\nFinancial Leverage (Reporting Currency) Actual Forecast\nFiscal Year, ends 31 Dec 2020 2021 2022 2023 2024 2025 2026 2027\nDebt/Capital % 49.3 70.0 58.3 55.3 53.6 51.2 48.8 47.2\nAssets/Equity 3.3 5.6 4.1 3.8 3.6 3.4 3.2 3.0\nNet Debt/EBITDA 2.8 3.5 9.6 2.7 2.6 2.4 2.3 2.1\nTotal Debt/EBITDA 3.9 4.4 10.1 3.1 3.1 2.9 2.8 2.7\nEBITDA/ Net Interest Expense 5.1 7.6 2.2 7.2 7.4 7.8 8.1 8.',
│   │   'filename': None,
│   │   'score': 0.5773178935050964
│   },
│   {
│   │   'content': 'UTC\nFinancial Leverage (Reporting Currency) Actual Forecast\nFiscal Year, ends 31 Dec 2020 2021 2022 2023 2024 2025 2026 2027\nDebt/Capital % 49.3 70.0 58.3 55.3 53.6 51.2 48.8 47.2\nAssets/Equity 3.3 5.6 4.1 3.8 3.6 3.4 3.2 3.0\nNet Debt/EBITDA 2.8 3.5 9.6 2.7 2.6 2.4 2.3 2.1\nTotal Debt/EBITDA 3.9 4.4 10.1 3.1 3.1 2.9 2.8 2.7\nEBITDA/ Net Interest Expense 5.1 7.6 2.2 7.2 7.4 7.8 8.1 8.',
│   │   'filename': None,
│   │   'score': 0.5773178935050964
│   }
]